In [ ]:
print("Federated Learning V2")

In [ ]:
import os
import numpy as np
import cv2 as cv
import tensorflow as tf

In [ ]:
datasets = ['/kaggle/input/lung-cancer-012/lcD-0/lcD-0',
            '/kaggle/input/lung-cancer-012/lcD-1/lcD-1/train',
            '/kaggle/input/lung-cancer-012/lcD-2/lcD-2/train']


In [ ]:
data_label={
    'Normal cases':0,
    'Bengin cases' :1,
    'Malignant cases':2,
}

In [ ]:
import random
client_dataset=[]
for dataset in datasets:
    data_dir = dataset
    images = []
    labels = []
    for img_class in os.listdir(data_dir):
      for img in os.listdir(os.path.join(data_dir,img_class)):
        img_path = os.path.join(data_dir, img_class, img)
        img_arr = cv.imread(img_path)
        resize_img = cv.resize(img_arr,(224,224))
        images.append(resize_img)
        labels.append(data_label[img_class])
    images = [value / 255.0 for value in images]
    images = np.array(images)
    labels = np.array(labels)
    client_dataset.append((images,labels))


In [ ]:
len(client_dataset)

In [ ]:
print(len(client_dataset[0][0]))
print(client_dataset[0][0].shape)

In [ ]:
print(client_dataset[0][0][1].shape)

In [ ]:
unique_values = set(client_dataset[2][1])
print(unique_values)

In [ ]:
import matplotlib.pyplot as plt
images = client_dataset[0][0][:9]
labels = client_dataset[0][1][:9]
fig = plt.figure(figsize=(12, 6))
for x in range(1, 9):
    fig.add_subplot(2, 4, x)
    plt.axis('off')
    plt.title(labels[x])
    plt.imshow(images[x])
plt.rcParams.update({'font.size': 20})
plt.show()

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D,MaxPool2D, Dense, Conv2D, Normalization, BatchNormalization, Flatten, Dropout, Input, Activation, Concatenate, Multiply, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import *

In [ ]:
from keras.applications.vgg16 import VGG16
w ='/kaggle/input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_vgg16 = VGG16(
    include_top=False,
    weights= w,
    input_shape=(224,224,3),
    classifier_activation="relu",
)
for layer in base_vgg16.layers:
  layer.trainable = False

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
def cbam_block(input_feature, ratio=8):
    # Channel Attention (CA) submodule
    x = input_feature
    batch, _, _, channel = x.shape
    
    ## Shared layers
    l1 = Dense(channel//ratio, activation="relu", use_bias=False)
    l2 = Dense(channel, use_bias=False)

    ## Global Average Pooling
    x1 = GlobalAveragePooling2D()(x)
    x1 = l1(x1)
    x1 = l2(x1)

    ## Global Max Pooling
    x2 = GlobalMaxPooling2D()(x)
    x2 = l1(x2)
    x2 = l2(x2)

    ## Add both the features and pass through sigmoid
    feats_ca = x1 + x2
    feats_ca = Activation("sigmoid")(feats_ca)
    feats_ca = Multiply()([x, feats_ca])

    # Spatial Attention (SA) submodule
    ## Average Pooling
    y1 = tf.reduce_mean(feats_ca, axis=-1)
    y1 = tf.expand_dims(y1, axis=-1)

    ## Max Pooling
    y2 = tf.reduce_max(feats_ca, axis=-1)
    y2 = tf.expand_dims(y2, axis=-1)

    ## Concatenat both the features
    feats_sa = Concatenate()([y1, y2])
    ## Conv layer
    feats_sa = Conv2D(1, kernel_size=7, padding="same", activation="sigmoid")(feats_sa)
    feats_sa = Multiply()([x, feats_sa])

    # Combine CA and SA
    cbam_feature = layers.Add()([feats_ca, feats_sa])

    return cbam_feature

In [ ]:
x = cbam_block(base_vgg16.layers[-1].output)
x = Flatten() (x)
x = Dense(526, activation= "relu") (x)
x = Dropout(rate=0.2)(x)

x = Dense(128, activation="sigmoid") (x)
x = Dense(64, activation="sigmoid") (x)
output = Dense(3, activation = 'softmax') (x)
vgg_16_cbam_model = Model(base_vgg16.input, output)

vgg_16_cbam_model.summary()

In [ ]:
from tensorflow import keras
keras.utils.plot_model(vgg_16_cbam_model, show_shapes=True)

In [ ]:
NUM_CLIENTS = 3
NUM_ROUNDS = 2

for round_num in range(NUM_ROUNDS):
    for i in range(NUM_CLIENTS):
        client_model = tf.keras.models.clone_model(vgg_16_cbam_model)
        client_model.set_weights(vgg_16_cbam_model.get_weights())
        client_model.compile(optimizer=Adam(learning_rate=0.0001),
        loss='sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy'])
        client_model.fit(x = client_dataset[i][0], y = client_dataset[i][1], batch_size= 32,
                         epochs=20, validation_split=0.2)
        
        # Aggregate the model
        new_weights = []
        for layer_index in range(len(vgg_16_cbam_model.get_weights())):
            new_layer_weights = np.mean([client_model.get_weights()[layer_index], vgg_16_cbam_model.get_weights()[layer_index]], axis=0)
            new_weights.append(new_layer_weights)
        vgg_16_cbam_model.set_weights(new_weights)

In [ ]:
vgg_16_cbam_model.save('bioInfo_modelV1.h5')

In [ ]:
test_path = '/kaggle/input/lung-cancer-test/test'
X_test =[]
Y_test =[]
for img_class in os.listdir(test_path):
    for img in os.listdir(os.path.join(test_path, img_class)):
        img_path = os.path.join(test_path, img_class, img)
        img_arr = cv.imread(img_path)
        resize_img = cv.resize(img_arr,(224,224))
        X_test.append(resize_img)
        if img_class == 'BenginCases':
            Y_test.append(1)
        elif img_class == 'normal':
            Y_test.append(0)
        else:
            Y_test.append(2)
X_test = np.array(X_test)        
Y_test = np.array(Y_test)

In [ ]:
X_test[1].shape

In [ ]:
input_data = X_test[1] 
input_data_with_batch = np.expand_dims(input_data, axis=0)
predictions = vgg_16_cbam_model.predict(input_data_with_batch)
print(Y_test[1])
print(predictions)

In [ ]:
p = []
for data in X_test:
    input_data = data
    input_data_with_batch = np.expand_dims(input_data, axis=0)
    predictions = vgg_16_cbam_model.predict(input_data_with_batch)
    pred = np.argmax(predictions, axis=-1)
    p.append(pred)


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, p))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

# Calculate confusion matrix
cm = confusion_matrix(Y_test, p)

# Create a heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d', xticklabels=['0','1','2'],
            yticklabels=['0','1','2'])
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix')
plt.show()